# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
# import dependencies
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Experiment, Dataset
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.data.dataset_factory import TabularDatasetFactory
#from azureml.train.hyperdrive import uniform, choice
from azureml.core import ScriptRunConfig
from azureml.core import Environment
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

import os
import shutil


## Initialize Workspace and Create an Azure ML experiment

In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

AZML
RG-AZML
eastus
a748a023-896b-4719-8f8e-3d1ba62d6e35


In [4]:
experiment_name = 'ideb-regression'

experiment=Experiment(ws, experiment_name)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
datastore = ws.get_default_datastore()
path = "data/data.csv"
try:
    ideb_dataset = Dataset.get_by_name(ws, name="ideb_dataset")
except:
    datastore.upload('data', target_path='data')
    # Create TabularDataset & register in workspace
    ideb_dataset = Dataset.Tabular.from_delimited_files([(datastore, path)])
    ideb_dataset = ideb_dataset.register(
        ws, name="ideb_dataset", create_new_version=True,
        description="Dataset for ideb prediction"
    )

## Creating Compute

In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "automl-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Jobrunning............................................................................................................................................................................................................................................................................................
Running


## Create an environment

In [7]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies


env = Environment('my-sklearn-environment')
env.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'scikit-learn==0.22.1'
])

In [8]:
with open('score.py') as f:
    print(f.read())

# ---------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# ---------------------------------------------------------
import json
import logging
import os
import pickle
import numpy as np
import pandas as pd
import joblib

import azureml.automl.core
from azureml.automl.core.shared import logging_utilities, log_server
from azureml.telemetry import INSTRUMENTATION_KEY

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType
@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))


input_sample = pd.DataFrame({"TAprov2019_Total": pd.Series([0.0], dtype="float64"), "TAprov2019_1_serie": pd.Series([0.0], dtype="float64"), "TAprov2019_2_serie": pd.Series([0.0], dtype="float64"), "TAprov2019_3_

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [9]:

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(
                        evaluation_interval=2,
                        slack_factor=0.1
)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( 
    {
         '--max_depth': choice(range(1, 20)),
         '--min_samples_split': choice(2, 5, 10, 15, 100),
         '--min_samples_leaf': choice(range(0, 10))
    }
)

#TODO: Create your estimator and hyperdrive config
#estimator = SKLearn(
#                source_directory="./",
#                entry_script="train.py",
#                compute_target=cpu_cluster_name
#)

src = ScriptRunConfig(source_directory='./',
                      script='train.py',
                      compute_target = compute_target,
                      environment=env)

hyperdrive_config = HyperDriveConfig(
                    primary_metric_name='accuracy',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=20,
                    max_concurrent_runs=4,
                    hyperparameter_sampling=param_sampling,
                    policy=early_termination_policy,
                    #estimator=estimator,
                    run_config=src
)


##  Submit Run

In [10]:
# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_c7b6659c-350b-4ba5-b458-602f77d160cf
Web View: https://ml.azure.com/experiments/ideb-regression/runs/HD_c7b6659c-350b-4ba5-b458-602f77d160cf?wsid=/subscriptions/a748a023-896b-4719-8f8e-3d1ba62d6e35/resourcegroups/RG-AZML/workspaces/AZML

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-24T12:55:06.402886][API][INFO]Experiment created<END>\n""<START>[2021-01-24T12:55:06.952209][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-24T12:55:07.151096][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-24T12:55:08.1011059Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_c7b6659c-350b-4ba5-b458-602f77d160cf
Web View: https://ml.azure.com/experiments/ideb-regression/runs/HD_c7b6659c-350b-4ba5-b458-602f77d160cf?wsid=/subscriptions/a748a023-896b-4719-8f8e-3d1ba62d6e35/r

{'runId': 'HD_c7b6659c-350b-4ba5-b458-602f77d160cf',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-24T12:55:06.187397Z',
 'endTimeUtc': '2021-01-24T13:08:36.595654Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name":

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [12]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
print(hyperdrive_run.get_metrics())

None
{'HD_c7b6659c-350b-4ba5-b458-602f77d160cf_19': {'max_depth:': 11, 'min_samples_split:': 10, 'min_samples_leaf:': 3, 'Accuracy': 0.997495616902193}, 'HD_c7b6659c-350b-4ba5-b458-602f77d160cf_16': {'max_depth:': 14, 'min_samples_split:': 10, 'min_samples_leaf:': 5, 'Accuracy': 0.9975108698894305}, 'HD_c7b6659c-350b-4ba5-b458-602f77d160cf_17': {'max_depth:': 2, 'min_samples_split:': 15, 'min_samples_leaf:': 0}, 'HD_c7b6659c-350b-4ba5-b458-602f77d160cf_18': {'max_depth:': 11, 'min_samples_split:': 10, 'min_samples_leaf:': 8, 'Accuracy': 0.9966121755513355}, 'HD_c7b6659c-350b-4ba5-b458-602f77d160cf_15': {'max_depth:': 9, 'min_samples_split:': 15, 'min_samples_leaf:': 8, 'Accuracy': 0.9965147659406796}, 'HD_c7b6659c-350b-4ba5-b458-602f77d160cf_14': {'max_depth:': 9, 'min_samples_split:': 10, 'min_samples_leaf:': 7, 'Accuracy': 0.9967485199113563}, 'HD_c7b6659c-350b-4ba5-b458-602f77d160cf_13': {'max_depth:': 15, 'min_samples_split:': 10, 'min_samples_leaf:': 1, 'Accuracy': 0.9976889609252

In [13]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics=best_run.get_metrics()
best_run_files=best_run.get_file_names()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run ID',best_run.id)
print('\n Metrics: ', best_run_metrics)
print('\n Parameters: ', parameter_values,sep='\n')
print('\nAccuracy of Best run',best_run_metrics['Accuracy'],sep='\n')
print('\nBest run file names',best_run_files,sep='\n')

AttributeError: 'NoneType' object has no attribute 'get_metrics'

## Save the best model

In [ ]:
#TODO: Save the best model
model = best_run.register_model(model_name='best-hyper-model', model_path='outputs/model.pkl')
print('Best model registered')
model


model.download(target_dir="outputs", exist_ok=True)
print('model downloaded')

In [ ]:
envs = Environment.list(workspace=ws)
for env in envs:
    if env.startswith("AzureML-AutoML"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

In [ ]:
myenv = remote_run.get_environment()

In [ ]:
os.listdir(os.curdir), os.getcwd()

In [ ]:
with open('score.py') as f:
    print(f.read())

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice

# Register the model to deploy
model = run.register_model(model_name = "best_hyper_model", model_path = "outputs/model.pkl", description = "Best model trained with Hyperdrive")

# Combine scoring script & environment in Inference configuration
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(
    workspace = ws,
    name = "ideb_prediction_service",
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config)
service.wait_for_deployment(show_output=True)
print(service.state)

In [ ]:
print(service.state)

In [ ]:
service.get_logs()

## Test the web service deployed

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

In [ ]:
import requests
import json

# scoring endpoint
scoring_uri = service.scoring_uri


# If the service is authenticated, set the key or token
#key = '<your key or token>'

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
           "Cod_Escola": 4,
           "TAprov2019_Total": 99.9, 
           "TAprov2019_1_serie": 99.2, 
           "TAprov2019_2_serie": 59.1, 
           "TAprov2019_3_serie": 60.5, 
           "TAprov2019_4_serie": 70.5, 
           "Indicador_Rendimento": 0.99, 
           "SAEB2019_Matematica": 365.38, 
           "SAEB2019_Lingua Portuguesa": 351.54, 
           "SAEB2019_Nota Media Padronizada": 7.055853, 
           "IDEB_2019": 7.1
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

## Print the logs of the web service and delete the service

In [ ]:
service.get_logs()
service.delete()